In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import FloatType

In [2]:
spark = SparkSession.builder.appName("L3Q2").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/11 14:23:37 WARN Utils: Your hostname, dbl-23, resolves to a loopback address: 127.0.1.1; using 172.16.58.182 instead (on interface eno1)
25/08/11 14:23:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 14:23:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df1 = spark.read.csv('datasets/Tokenized.csv/part-00000-8ad89596-b847-43fc-b712-ec12acf2c8ce-c000.csv', header=True)
df2 = spark.read.csv('datasets/Tokenized.csv/part-00001-8ad89596-b847-43fc-b712-ec12acf2c8ce-c000.csv', header=True)

df = df1.union(df2)


In [4]:
df = df.withColumn('tokens', split('tokens', ','))
df_pairs = df.alias("a").crossJoin(df.alias("b"))

In [5]:
def jaccard_similarity(tokens1, tokens2):
    set1 = set(str(tokens1))
    set2 = set(str(tokens2))
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0
    return float(len(intersection)) / len(union)

In [6]:
jaccard_udf = udf(jaccard_similarity)

In [7]:
result = df_pairs.withColumn("jaccard_similarity",jaccard_udf(col("a.tokens"), col("b.tokens"))).select(col('a.id').alias('id_1'), col('b.id').alias('id_2'),col('jaccard_similarity'))
result.show(5)

+----+----+-------------------+
|id_1|id_2| jaccard_similarity|
+----+----+-------------------+
|1859|1859|                1.0|
|1859|1402| 0.2857142857142857|
|1859|3503| 0.5555555555555556|
|1859|1009| 0.4444444444444444|
|1859| 276|0.42105263157894735|
+----+----+-------------------+
only showing top 5 rows


In [10]:
## for usage of Similarity in 3rd Question
result.write.csv('datasets/jaccard_similarity', header=True)

In [11]:
spark.stop()